# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
if geoapify_key=="YOUR KEY HERE!":
    print(f"Please configure your config.py file")

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,-9.67,73,0,4.00,RU,1731167565
1,1,bethel,41.3712,-73.4140,9.53,42,0,4.12,US,1731167626
2,2,port-aux-francais,-49.3500,70.2167,5.04,88,77,9.25,TF,1731167559
3,3,adamstown,-25.0660,-130.1015,22.82,88,100,7.27,PN,1731167549
4,4,katsuura,35.1333,140.3000,17.28,85,99,5.68,JP,1731167820


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Configure the map plot
map_plot_humid = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.8,
    hover_cols = "City"
)

# Display the map
map_plot_humid

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [85]:
# Narrow down cities that fit criteria and drop any results with null values
dream_temp_min = 25
dream_temp_max = 35
dream_humid_max = 75
dream_wind_max = 4.5
dream_clouds_max = 20

dream_city_df = city_data_df.loc[city_data_df["Max Temp"] > dream_temp_min]
dream_city_df = dream_city_df.loc[dream_city_df["Max Temp"] < dream_temp_max]
dream_city_df = dream_city_df.loc[dream_city_df["Humidity"] < dream_humid_max]
dream_city_df = dream_city_df.loc[dream_city_df["Wind Speed"] < dream_wind_max]
dream_city_df = dream_city_df.loc[dream_city_df["Cloudiness"] < dream_clouds_max]

# Drop any rows with null values
dream_city_df.dropna()

# Display sample data
dream_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,mangrol,21.1167,70.1167,28.46,67,2,2.77,IN,1731168041
78,78,mercedes,-34.6515,-59.4307,27.75,25,4,3.93,AR,1731168122
104,104,mankera,31.3866,71.4407,26.11,30,0,2.34,PK,1731168151
220,220,qina,26.1642,32.7267,25.74,31,0,3.98,EG,1731168286
222,222,tazacorte,28.6290,-17.9293,25.54,69,0,2.06,ES,1731168288


### Step 3: Create a new DataFrame called `hotel_df`.

In [81]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = dream_city_df.copy(deep=True)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "NaN"

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,mangrol,IN,21.1167,70.1167,67,NaN
54,ad dilam,SA,23.9915,47.1627,27,NaN
78,mercedes,AR,-34.6515,-59.4307,25,NaN
102,touros,BR,-5.1989,-35.4608,59,NaN
104,mankera,PK,31.3866,71.4407,30,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [82]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "limit": 20,
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mangrol - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
mercedes - nearest hotel: Gran Hotel Mercedes
touros - nearest hotel: Pousada Atlântico
mankera - nearest hotel: No hotel found
totoras - nearest hotel: Raíces Apart
qina - nearest hotel: Basma Hotel
tazacorte - nearest hotel: App Leyma
howrah - nearest hotel: Sun India Guest House
rocha - nearest hotel: Residence Inn
kidal - nearest hotel: No hotel found
teixeira - nearest hotel: Hotel da Serra
san vicente - nearest hotel: Los Lapachos
carutapera - nearest hotel: No hotel found
mananjary - nearest hotel: No hotel found
daulatpur - nearest hotel: No hotel found
puerto san jose - nearest hotel: Martita
ifaki - nearest hotel: No hotel found
kotri - nearest hotel: Shah Ji Hotel
kopaganj - nearest hotel: No hotel found
buenos aires - nearest hotel: Hotel City Baires
cabo san lucas - nearest hotel: Comfort Rooms
madaoua - nearest hotel: No hotel found
tinajo - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
11,mangrol,IN,21.1167,70.1167,67,No hotel found
54,ad dilam,SA,23.9915,47.1627,27,No hotel found
78,mercedes,AR,-34.6515,-59.4307,25,Gran Hotel Mercedes
102,touros,BR,-5.1989,-35.4608,59,Pousada Atlântico
104,mankera,PK,31.3866,71.4407,30,No hotel found
141,totoras,AR,-32.5844,-61.1685,23,Raíces Apart
220,qina,EG,26.1642,32.7267,31,Basma Hotel
222,tazacorte,ES,28.6290,-17.9293,69,App Leyma
226,howrah,IN,22.5892,88.3103,74,Sun India Guest House
240,rocha,UY,-34.4833,-54.3333,38,Residence Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [83]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 20,
    hover_cols = ["City", "Humidity", "Hotel Name", "Country"],
    color = "City"
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [84]:
#Creating another map with only the cities where a hotel was found

clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]

map_plot_hotels_clean = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 20,
    hover_cols = ["City", "Humidity", "Hotel Name", "Country"],
    color = "City"
)
map_plot_hotels_clean

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)